In [18]:
# To run this file correctly, do the following
# indent : entire code inside to function
# delete : is_only_admin=0  and For Dev psycopg2 and dotnet env
# uncomment: For Production running on Python Enviroment
# uncomment : return and dtNow=datetime.now()

In [19]:
# def notify_imcomplete_pm_to_team(is_only_admin):

In [20]:
import pandas as pd
from dotenv import dotenv_values

from datetime import datetime,date
from dateutil import relativedelta
import os


# Paramter & Config values

In [21]:
temp_doc="temp_pm_notifcation"

# Retrive data from SMartDB Postgresql


In [22]:
# For Production running on Python Enviroment
# from django.db import connection
# def get_postgres_conn():
#   return connection


# For Dev
env_path=r'D:\PythonDev\Yit\SMartApp_ScriptDev\.env'
config = dotenv_values(dotenv_path=env_path)

import psycopg2
import psycopg2.extras as extras
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error


In [23]:
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

# Get the next month 
* Set Window Sheduler to monthly run at the fist dsate of The month
* No matter what day you run this job , the program will  get only all PMs over the current month

In [30]:
#dtNow= datetime.strptime(datetime(2023,12,1,6,0).strftime('%Y-%m-%d'),'%Y-%m-%d')
dtNow=datetime.now()

dt = datetime.strptime(dtNow.strftime('%Y-%m-%d'),'%Y-%m-%d')
print(dt)

first_day_next_month = dt + relativedelta.relativedelta(months=1, day=1)

print(first_day_next_month ) 


2023-11-30 00:00:00
2023-12-01 00:00:00


# Retrive  and Transaform data

In [25]:
def do_something_df(df):
    df = df.fillna(value='')
    df = df.reset_index(drop=False)
    df.insert(0, "No", df["index"]+1,True)
    df=df.drop(columns=["index"])
    return df

In [33]:
sql_item = f"""
 select  ac.company_full_name as "ชื่อลูกค้า",
 ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" , ap.project_name as "ชื่อโครงการ",

             ai.serial_number as "Serial",
             (select  productype_name from app_product_type where id=ai.product_type_id ) as "ProudctType",
             (select  brand_name from app_brand where id=ai.brand_id ) as "Brand",
              (select  model_name from app_model where id=ai.model_id ) as "Model",

             TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
             pm.remark as  "งวดPM",
             ae.employee_name as "หัวหน้าทีม",
             (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Planed Engineer",

           (select employee_name from app_employee eng_pm  where eng_pm.id=pm_item.pm_engineer_id ) as "Operation Engineer",
           TO_CHAR(pm_item.actual_date,'DD Mon YYYY') as "ActualDate",pm_item.call_number as "Call Number",

            (select employee_name from app_employee eng_doc  where eng_doc.id=pm_item.document_engineer_id ) as "Doc Engineer",
           TO_CHAR(pm_item.document_date,'DD Mon YYYY') as "DocumentDate",pm_item.pm_document_number as "Doc Number",
           pm_item.remark as "Remark"
           
           ,ae.employee_name as "หัวหน้าทีม" 
           ,ae.employee_email as "email_teamlead"

from app_pm_inventory as pm_item
left join app_inventory ai on ai.id = pm_item.inventory_id
-- inner join app_product_type  product_type on ai.product_type_id = product_type.id
left join app_preventivemaintenance pm on pm.id = pm_item.pm_master_id
left join app_project ap on ap.id = pm.project_id
left join app_company ac on ac.id = ap.company_id
left join app_employee ae on ae.id =pm.team_lead_id

where pm_item.is_pm=True and   pm.ended_pm_date<='{first_day_next_month}'
and
( 
  pm_item.actual_date is null or pm_item.document_date is null
  or ( pm_item.call_number is null  or pm_item.call_number<>''  ) 
  or ( pm_item.pm_document_number is null  or pm_item.pm_document_number<>'' )
  or ( pm_item.remark is null or pm_item.remark<>'' ) 
) 


order by  ac.company_full_name,ap.enq_id,pm.remark

    """

print(sql_item)


 select  ac.company_full_name as "ชื่อลูกค้า",
 ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" , ap.project_name as "ชื่อโครงการ",

             ai.serial_number as "Serial",
             (select  productype_name from app_product_type where id=ai.product_type_id ) as "ProudctType",
             (select  brand_name from app_brand where id=ai.brand_id ) as "Brand",
              (select  model_name from app_model where id=ai.model_id ) as "Model",

             TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
             pm.remark as  "งวดPM",
             ae.employee_name as "หัวหน้าทีม",
             (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Planed Engineer",

           (select employee_name from app_employee eng_pm  where eng_pm.id=pm_item.pm_engineer_id ) as "Operation Engineer",
           TO_CHAR(pm_item.actual_date,'DD Mon YYYY') as "ActualDate",pm_item.call_number 

In [34]:
dfItem=list_data(sql_item,None,get_postgres_conn())
dfItem=do_something_df(dfItem)
dfItem.info()
dfItem.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20047 entries, 0 to 20046
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   No                  20047 non-null  int64 
 1   ชื่อลูกค้า          20047 non-null  object
 2   เลขที่สัญญา         20047 non-null  object
 3   ENQ                 20047 non-null  object
 4   ชื่อโครงการ         20047 non-null  object
 5   Serial              20047 non-null  object
 6   ProudctType         20047 non-null  object
 7   Brand               20047 non-null  object
 8   Model               20047 non-null  object
 9   แผนจะทำPM           20047 non-null  object
 10  วันสุดท้ายที่ทำPM   20047 non-null  object
 11  งวดPM               20047 non-null  object
 12  หัวหน้าทีม          20047 non-null  object
 13  Planed Engineer     20047 non-null  object
 14  Operation Engineer  20047 non-null  object
 15  ActualDate          20047 non-null  object
 16  Call Number         20

,No,ชื่อลูกค้า,เลขที่สัญญา,ENQ,ชื่อโครงการ,Serial,ProudctType,Brand,Model,แผนจะทำPM,...,หัวหน้าทีม,Planed Engineer,Operation Engineer,ActualDate,Call Number,Doc Engineer,DocumentDate,Doc Number,Remark,email_teamlead
0,1,Advanced Info Services Public Company Limited,,ENQ/62-320-1,AIS-ติดตั้ง Server Rack DL560 G10 จำนวน 8 เครื...,SGH932V6SQ,Server,HPE,ProLiant DL560 Gen10,Apr 2023,...,Boonlert Kawta,,Panusak Sornplang,03 Apr 2023,"15361, 15523",Paradon Srimode,03 Apr 2023,,,boonlert.ka@yipintsoi.com
1,2,Advanced Info Services Public Company Limited,,ENQ/62-320-1,AIS-ติดตั้ง Server Rack DL560 G10 จำนวน 8 เครื...,SGH932V6S6,Server,HPE,ProLiant DL560 Gen10,Apr 2023,...,Boonlert Kawta,,Panusak Sornplang,03 Apr 2023,"15361, 15523",Paradon Srimode,03 Apr 2023,,,boonlert.ka@yipintsoi.com
2,3,Advanced Info Services Public Company Limited,,ENQ/62-320-1,AIS-ติดตั้ง Server Rack DL560 G10 จำนวน 8 เครื...,SGH932V6SB,Server,HPE,ProLiant DL560 Gen10,Apr 2023,...,Boonlert Kawta,,Panusak Sornplang,03 Apr 2023,"15361, 15523",Paradon Srimode,03 Apr 2023,,,boonlert.ka@yipintsoi.com
3,4,Advanced Info Services Public Company Limited,,ENQ/62-320-1,AIS-ติดตั้ง Server Rack DL560 G10 จำนวน 8 เครื...,SGH932V6SL,Server,HPE,ProLiant DL560 Gen10,Apr 2023,...,Boonlert Kawta,,Panusak Sornplang,03 Apr 2023,"15361, 15523",Paradon Srimode,03 Apr 2023,,,boonlert.ka@yipintsoi.com
4,5,Advanced Info Services Public Company Limited,,ENQ/62-320-1,AIS-ติดตั้ง Server Rack DL560 G10 จำนวน 8 เครื...,SGH932V6T3,Server,HPE,ProLiant DL560 Gen10,Apr 2023,...,Boonlert Kawta,,Panusak Sornplang,03 Apr 2023,"15361, 15523",Paradon Srimode,03 Apr 2023,,,boonlert.ka@yipintsoi.com


# Excel file

In [ ]:
with pd.ExcelWriter(file_path) as writer:
    dfPM.to_excel(writer, sheet_name="PM-Plan",index=False)
    dfItem.to_excel(writer, sheet_name="PM-Item",index=False)
    print(f"Exported {file_name} file for email successfully.")

# Email Office 365

# Delete attach files : if sent email with excel file completely

In [ ]:

os.remove(f"{file_path}")

In [ ]:
# return True